## Ethanol yield from glucose

https://groups.google.com/forum/#!topic/cobra-pie/-sPHjNsB974

This is definitely doable. 

Steps are:
1. Get a genome scale metabolic model for yeast such as iMM904. http://bigg.ucsd.edu/ has it.
2. [hardest part - should take a beginner about 1 hour depending on molecule] Add reactions needed to make your molecule and a "demand reaction" for it to leave the cell.  Decide if you want to account for transport costs or not.
3. Optimize (maximize) flux just to this demand reaction. This flux is the molar yield assuming you provided 1 mmol glucose gDW-1 h-1 and have no other demands (turn off ATP maintenance reaction, etc.)
4. Convert to a g/g yield basis as is typical using the molecular weights of glucose and your product.
5. Verify overall stoichiometry is plausible (look at all the exchanges of what is going in and out of the cell in the max yield simulation)

I'm happy to peer review your progress!  Perhaps write back with the Ethanol max yield?  That is easily doable as the molecule is native and you can skip #2.
https://cobrapy.readthedocs.io/en/latest/simulating.html#  for the basics. 

Good luck!
Josh


----

Hi Björn,

In addition to what Josh said, take a look at the following materials:

https://cobrapy.readthedocs.io/en/latest/phenotype_phase_plane.html

Our general materials for the software carpentry course on cell design:
https://biosustain.github.io/cell-factory-design-course/

In particular:
https://biosustain.github.io/cell-factory-design-course/08-Predict-heterologous-pathways/

https://biosustain.github.io/cell-factory-design-course/07-Theoretical-maximum-yields/

Also take a look at the documentation for cameo, our strain design library
https://cameo.readthedocs.io/en/latest/

Happy to answer any specific question and congrats to your acquiring a very experienced peer reviewer ;)




In [1]:
import cameo

In [2]:
from cameo import models
models.index_models_bigg()

,bigg_id,gene_count,metabolite_count,organism,reaction_count
0,e_coli_core,137,72,Escherichia coli str. K-12 substr. MG1655,95
1,iAB_RBC_283,346,342,Homo sapiens,469
2,iAF1260,1261,1668,Escherichia coli str. K-12 substr. MG1655,2382
3,iAF1260b,1261,1668,Escherichia coli str. K-12 substr. MG1655,2388
4,iAF692,692,628,Methanosarcina barkeri str. Fusaro,690
5,iAF987,987,1109,Geobacter metallireducens GS-15,1285
6,iAPECO1_1312,1313,1942,Escherichia coli APEC O1,2735
7,iAT_PLT_636,636,738,Homo sapiens,1008
8,iB21_1397,1337,1943,Escherichia coli BL21(DE3),2741
9,iBWG_1329,1329,1949,Escherichia coli BW2952,2741


In [3]:
iMM904 = models.bigg.iMM904
model = iMM904.copy()

In [4]:
model.compartments

{'c': 'cytosol',
 'm': 'mitochondria',
 'e': 'extracellular space',
 'x': 'peroxisome/glyoxysome',
 'r': 'endoplasmic reticulum',
 'v': 'vacuole',
 'n': 'nucleus',
 'g': 'golgi apparatus'}

In [5]:
for reaction in model.reactions:
    print(reaction.id,"                    ",reaction.name)

13BGH                      Endo 1 3 beta glucan glucohydrase
13BGHe                      Exo 1 3 beta glucan glucohydrase
13GS                      1 3 beta glucan synthase
16GS                      1 6 beta glucan synthase
23CAPPD                      ADP ribose 2 3 cyclic phosphodiesterase
2DDA7Ptm                      2 Dehydro 3 deoxy D arabino heptonate7 phohsphate mitochondrial transport via diffusion
2DHPtm                      2 Dehydropantoate mitochondrial transport
2DOXG6PP                      2 deoxyglucose 6 phosphatase
2HBO                      2 HydroxybutyrateNAD oxidoreductase
2HBt2                      2 hydroxybutyrate cotransport with proton
2HMHMBQMTm                      2 hexaprenyl 3 methyl 5 hydroxy 6 methoxy 1 4 benzoquinone methyltransferase  mitochondrial
2HP6MPMOm                      2 Hexaprenyl 6 methoxyphenol monooxygenase  mitochondrial
2HPMBQMTm                      2 hexaprenyl 6 methoxy 1 4 benzoquinone methyltransferase  mitochondrial
2HPMMBQMOm  

model.reactions.EX_glc__D_e

In [6]:
model.reactions.ALCD2x_copy1

Reaction identifier,ALCD2x_copy1
Name,Alcohol dehydrogenase (ethanol)
Memory address,0x07ff67f821ac8
Stoichiometry,etoh_c + nad_c --> acald_c + h_c + nadh_c Ethanol + Nicotinamide adenine dinucleotide --> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YMR303C
Lower bound,0.0
Upper bound,999999.0


In [7]:
model.reactions.ALCD2x_copy2

Reaction identifier,ALCD2x_copy2
Name,Alcohol dehydrogenase (ethanol)
Memory address,0x07ff67f821e80
Stoichiometry,etoh_c + nad_c <=> acald_c + h_c + nadh_c Ethanol + Nicotinamide adenine dinucleotide <=> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YDL168W
Lower bound,-999999.0
Upper bound,999999.0


In [8]:
model.reactions.ALCD2ir

Reaction identifier,ALCD2ir
Name,Alcohol dehydrogenase reverse rxn acetaldehyde ethanol
Memory address,0x07ff67f821e10
Stoichiometry,acald_c + h_c + nadh_c --> etoh_c + nad_c Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced --> Ethanol + Nicotinamide adenine dinucleotide
GPR,YOL086C or YGL256W or YBR145W
Lower bound,0.0
Upper bound,999999.0


In [9]:
model.reactions.EX_etoh_e

Reaction identifier,EX_etoh_e
Name,Ethanol exchange
Memory address,0x07ff67f8735c0
Stoichiometry,etoh_e --> Ethanol -->
GPR,
Lower bound,0.0
Upper bound,999999.0


In [10]:
print(model.objective)

Maximize
1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090


In [11]:
model.objective = model.reactions.EX_etoh_e

In [12]:
print(model.objective)

Maximize
1.0*EX_etoh_e - 1.0*EX_etoh_e_reverse_cc64f


In [13]:
solution = model.optimize()

In [14]:
solution.fluxes.EX_etoh_e

19.999999999999996

In [15]:
solution.fluxes.EX_glc__D_e

-10.0

In [16]:
(-1. * solution.fluxes.EX_etoh_e) / solution.fluxes.EX_glc__D_e

1.9999999999999996

2 moles of ethanol can be made from 1 mole of glucose so this seems about right.

In [29]:
solution

,fluxes,reduced_costs
13BGH,0.000000e+00,0.000000e+00
13BGHe,0.000000e+00,-4.440892e-16
13GS,0.000000e+00,0.000000e+00
16GS,0.000000e+00,-4.000000e+00
23CAPPD,0.000000e+00,-1.480942e-20
...,...,...
ZYMSTAT_SC,0.000000e+00,-1.666667e-02
ZYMSTESTH_SC,1.345200e-14,1.582935e-17
ZYMSTESTH_SCe,0.000000e+00,1.008308e-17
ZYMSTt,0.000000e+00,1.541081e-16


In [34]:
solution.

AttributeError: 'Solution' object has no attribute 'display_on_map'

In [35]:
import escher

In [40]:
escher.Builder('iMM904.Central carbon metabolism').display_in_notebook()
#escher.Builder('iMM904.Central carbon metabolism', solution).display_in_notebook()